In [ ]:
import cv2
import os
import pandas as pd
from PIL import Image
import numpy as np
import re
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
img = np.load('/content/drive/MyDrive/EECS545 Final Project/original_size_images.npy', allow_pickle=True)

In [ ]:
maxh = 310
maxw = 360

In [ ]:
nna_idx = np.load('/content/drive/MyDrive/EECS545 Final Project/nna_idx.np.npy')

In [ ]:
with open('filename.txt', 'r') as file:
    fn = file.readlines()
    fn = [line.strip() for line in fn]

In [ ]:
print(sum(nna_idx))

In [ ]:
img2_5D = np.zeros((len(nna_idx), 3, maxh, maxw), dtype = int)

In [ ]:
def pad_image(img_arr, height, width):
    # Calculate the amount of padding needed
    h, w = img_arr.shape[:2]
    pad_h = max(0, height - h)
    pad_w = max(0, width - w)

    # Add padding to the image
    padded_arr = np.pad(img_arr, ((0, pad_h), (0, pad_w)), mode='constant', constant_values=0)
    return(padded_arr)


In [ ]:
for i in range(len(nna_idx)):
    if nna_idx[i]:
#         print(i)
        pre_sn = re.search(r"(\w+_\w+)_slice", fn[i-1]).group(1)
        curr_sn = re.search(r"(\w+_\w+)_slice", fn[i]).group(1)
        next_sn = re.search(r"(\w+_\w+)_slice", fn[i+1]).group(1)
        img2_5D[i,1,:,:] = pad_image(img[i], maxh, maxw)
        if(pre_sn.lower() == curr_sn.lower()):
            img2_5D[i,0,:,:] = pad_image(img[i-1], maxh, maxw)
        if(next_sn.lower() == curr_sn.lower()):
            img2_5D[i,2,:,:] = pad_image(img[i+1], maxh, maxw)
#         print(pad_image(img[i], maxh, maxw).shape)


In [ ]:
res_img = img2_5D[nna_idx,:,:,:]

In [ ]:
np.save("images_25D.npy", res_img)

In [ ]:
stom = np.load('stom_seg.npy', allow_pickle=True)
sb = np.load('small_bowel_seg.npy', allow_pickle=True)
lb = np.load('large_bowel_seg.npy', allow_pickle=True)

In [ ]:
full_seg_label = np.zeros((sum(nna_idx), 3, maxh, maxw), dtype = int)

In [ ]:
newnnanidx = np.zeros(len(nna_idx))

In [ ]:
j = 0
for i in range(len(nna_idx)):
    if not np.isnan(stom[i]).all():
        newnnanidx[i] = True
        full_seg_label[j,0,:,:] = pad_image(stom[i], maxh, maxw)

    if not np.isnan(sb[i]).all():
        newnnanidx[i] = True
        full_seg_label[j,1,:,:] = pad_image(sb[i], maxh, maxw)

    if not np.isnan(lb[i]).all():
        newnnanidx[i] = True
        full_seg_label[j,2,:,:] = pad_image(lb[i], maxh, maxw)
    if newnnanidx[i]:
        j = j + 1

In [ ]:
np.save("full_seg_labels.npy", full_seg_label)

In [ ]:
print(full_seg_label.shape)

(16590, 3, 310, 360)
